# Team DatenWelle

## Keyword merging with FuzzyWuzzy

This notebook loads the data from JSON format and performs some keyword cleaning and merging misspelled duplicates with fuzzyWuuzy package 

In [1]:
#!git pull
#!git status
#!git add 2-anya-keywords_FuzzyWuzzy.ipynb #1-anya_exploratory_analysis.ipynb
#!git commit -m 'added a thing to put keywords back into the dataframe (incomplete)'
#!git push
#!pip install -r ../requirements.txt

#after installed new libraries
#!pip freeze > requirements.txt
#!git add requirements.txt 
#!git add out_dedupl_100323.csv test.csv
#!git commit -m 'added output files f fuzzy wuzzy dedupe'
#!git commit -m 'added library fuzzywuzzy'
#!git push 

In [2]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy.process import dedupe
import functools


In [3]:
# Opening JSON file
f = open('../data/raw/CMS_2010_to_June_2022_ENGLISH.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

df = pd.DataFrame.from_dict(data)

#print(df.head())

#### Finding the subset of the data for 1 Jan 2019 - 1 Jan 2020 based on lastModifiedDate

In [4]:
df = df.sort_values(by='lastModifiedDate') #sort dataframe

datetimes = pd.to_datetime(df['lastModifiedDate'])
df['ts_lastModifiedDate']=datetimes
#df.iloc[ts_start]['ts_lastModifiedDate']

#find start index for subset 2019-2022
ts_start=datetimes[(datetimes > pd.Timestamp(year=2019, month=1, day=1).tz_localize('utc')) 
          & (datetimes < pd.Timestamp(year=2019, month=1, day=2).tz_localize('utc'))].min()
print(ts_start)
#find end date for subset 2019-2022
ts_end=datetimes[(datetimes > pd.Timestamp(year=2022, month=1, day=1).tz_localize('utc')) 
          & (datetimes < pd.Timestamp(year=2022, month=1, day=2).tz_localize('utc'))].min()
print(ts_end)

start_date=datetimes[datetimes == ts_start]
end_date=datetimes[datetimes == ts_end]

#find index for the chosen start and end dates
start_index=start_date.index[0]
print(start_index)
df[df.index == start_date.index[0]]

end_index=end_date.index[0]
print(end_index)
df[df.index == end_date.index[0]]

df_subset=df[start_index:end_index]


2019-01-01 03:57:28.904000+00:00
2022-01-01 02:35:51.098000+00:00
60278
150367


In [5]:
#df_subset=df_subset[:100]

In [6]:
len(df_subset)

33830

### Keywords exploration


In [7]:
df['keywords'] # is keywords in dictionary

55250     [{'name': 'Lisbon Treaty'}, {'name': 'European...
55251     [{'name': 'Berlin'}, {'name': 'New Year'}, {'n...
55252     [{'name': 'Haile Gebrselassie'}, {'name': 'Eth...
55253     [{'name': 'four hills tournament'}, {'name': '...
55254     [{'name': 'human rights'}, {'name': 'demonstra...
                                ...                        
175606    [{'name': 'Women's Euro 2022'}, {'name': 'Ada ...
175333    [{'name': 'Ferda Ataman'}, {'name': 'Federal A...
175380    [{'name': 'documentary'}, {'name': 'hate'}, {'...
175568    [{'name': 'abortion'}, {'name': 'Roe vs. Wade'...
175179    [{'name': 'documentary'}, {'name': 'pollution'...
Name: keywords, Length: 175659, dtype: object

In [8]:
df['keywords'].isna().sum()


0

In [9]:
#create series of keywords sets
def get_keywords(row):
    if row is None:
        return None
    else:
        res_set = set()
        for name_dict in row:
            res_set.add(name_dict['name'])
        return res_set

df['keywords'].apply(get_keywords)

55250     {Martin Schulz, European Union, Frederik Reinf...
55251     {2010, Brandenburg Gate, Berlin, Right Said Fr...
55252                 {Trier, Haile Gebrselassie, Ethiopia}
55253     {four hills tournament, simon ammann, garmisch...
55254     {moscow helsinski group, sakharov prize, human...
                                ...                        
175606    {Lena Oberdorf, Ada Hegerberg, Vivianne Miedem...
175333    {discrimination, cancel culture, Federal Anti-...
175380    {documentary, anti-Judaism, hate, antisemitism...
175568    {abortion, menstruation, Roe vs. Wade, privacy...
175179    {documentary, Sweden, toxic waste, pollution, ...
Name: keywords, Length: 175659, dtype: object

In [10]:
#extract individual keywords from the sets of sets

# should work but it is very slow for now with current gpus
# 10000 articles in 7 seconds
# df_subset (90090 articles) runs in 10 minutes 10 seconds

# sets=df_subset['keywords'].apply(get_keyword1) #full dataset
sets=df_subset['keywords'].apply(get_keywords)  #2019-2021 subset
#sets=sets[0:10000] #10000 articles
 
kw=functools.reduce(set.union, sets)

In [56]:
kw

{'malware',
 'state repression',
 'climate consensus',
 'Canary Islands',
 'Angela I',
 'Evan Rachel Wood',
 'used cars',
 'woodcarving',
 'taboo',
 'Grüne Woche',
 'Dugong',
 'Qassem Soleimani',
 'Food Truck',
 'Israel lockdown',
 'Philippe Monguillot',
 'Boko haram',
 'Capitol Violence',
 'Benedikt Taschen',
 'Charlemagne Prize',
 'Yahya Jammeh',
 'Middle Ages',
 'InternetLab',
 'Host',
 'Mosambique',
 'child vaccinations',
 'Abel prize',
 'Thierry Breton',
 'work-life balance',
 'Mossad',
 'digital contact tracing',
 'Air Traffic Control',
 'Omar Farouq',
 'Tyrannosaurus rex',
 'Israeli Supreme Court',
 'Jean Paul Gaultier',
 'steel',
 'Mikhael Gorbachev',
 'Krzyzowa',
 'Domestic violence',
 'Moise',
 'global warming.',
 'forest management',
 'Vatican Apostolic Archive',
 'myoelectric',
 'Italian Democratic Party',
 'Dominican Republic',
 'Verbot',
 'Robert Zoellick',
 'Bouteflika',
 'Fight',
 'banknotes',
 'De Telegraaf',
 'Pipe Shelf',
 'learning',
 'Muhyiddin Yassin',
 'National 

In [12]:
# # another way to extract individual keywords from the sets of sets that doesn't crash kernel is interrupted
# # so it might be more stable when later applied to the entire dataset

# #runs for 10000 articles in 5 seconds
# #runs for df_subset in  11 min 16 sec 

# sets=df_subset['keywords'].apply(get_keywords)
# #sets=sets[0:10000]
# from tqdm import tqdm
# def get_unique_keywords(sets):
#     result_set = set()
#     for row_set in tqdm(sets.values):
#         #result_set.union(row_set)
#         result_set = result_set.union(row_set)
#     return result_set

# unique_keywords = get_unique_keywords(sets)



In [14]:
#write the set of DW keywords before fuzzyWuzzy into the file
# pd.Series(list(unique_keywords)).to_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv')
pd.Series(list(kw)).to_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv')

In [15]:
#unique_keywords=kw

In [16]:
# #load set of DW keywords before fuzzyWuzzy into the file
# uni_kw=pd.read_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv') 

In [17]:
unique_keywords=set(uni_kw['0'])

In [57]:
unique_keywords

In [19]:
# sets_10000=sets

### Playing with FuzzyWuzzy

In [20]:
# #function from tutorial to get simplest matching ratio

# Str1 = "Apple Inc."
# Str2 = "apple Inc"
# Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
# print(Ratio)

In [21]:
help(process)

Help on module fuzzywuzzy.process in fuzzywuzzy:

NAME
    fuzzywuzzy.process - # encoding: utf-8

FUNCTIONS
    dedupe(contains_dupes, threshold=70, scorer=<function token_set_ratio at 0x7fb8441024d0>)
        This convenience function takes a list of strings containing duplicates and uses fuzzy matching to identify
        and remove duplicates. Specifically, it uses the process.extract to identify duplicates that
        score greater than a user defined threshold. Then, it looks for the longest item in the duplicate list
        since we assume this item contains the most entity information and returns that. It breaks string
        length ties on an alphabetical sort.
        
        Note: as the threshold DECREASES the number of duplicates that are found INCREASES. This means that the
            returned deduplicated list will likely be shorter. Raise the threshold for fuzzy_dedupe to be less
            sensitive.
        
        Args:
            contains_dupes: A list of st

In [22]:
help(fuzz.token_set_ratio)

Help on function token_set_ratio in module fuzzywuzzy.fuzz:

token_set_ratio(s1, s2, force_ascii=True, full_process=True)



In [53]:
unique_keywords


TypeError: pop() takes no arguments (1 given)

In [58]:
#len(ded_kw)
unique_keywords
list(unique_keywords)[:1000]

['malware',
 'state repression',
 'climate consensus',
 'Canary Islands',
 'Angela I',
 'Evan Rachel Wood',
 'used cars',
 'woodcarving',
 'taboo',
 'Grüne Woche',
 'Dugong',
 'Qassem Soleimani',
 'Food Truck',
 'Israel lockdown',
 'Philippe Monguillot',
 'Boko haram',
 'Capitol Violence',
 'Benedikt Taschen',
 'Charlemagne Prize',
 'Yahya Jammeh',
 'Middle Ages',
 'InternetLab',
 'Host',
 'Mosambique',
 'child vaccinations',
 'Abel prize',
 'Thierry Breton',
 'work-life balance',
 'Mossad',
 'digital contact tracing',
 'Air Traffic Control',
 'Omar Farouq',
 'Tyrannosaurus rex',
 'Israeli Supreme Court',
 'Jean Paul Gaultier',
 'steel',
 'Mikhael Gorbachev',
 'Krzyzowa',
 'Domestic violence',
 'Moise',
 'global warming.',
 'forest management',
 'Vatican Apostolic Archive',
 'myoelectric',
 'Italian Democratic Party',
 'Dominican Republic',
 'Verbot',
 'Robert Zoellick',
 'Bouteflika',
 'Fight',
 'banknotes',
 'De Telegraaf',
 'Pipe Shelf',
 'learning',
 'Muhyiddin Yassin',
 'National 

In [59]:
## !!VERY SLOW!!! 
# Took 186 minutes to run for 10000 articles

#fuzzy.process.dedupe function returns a list without duplicates. by default it is using 70% similarity ratio
#to explore similarity ratio for individual words use fuzzy.process.extract i.e. process.extract('angela merkel',unique_keywords,limit=20)

print(len(unique_keywords))
#ded_kw=dedupe(unique_keywords)
#ded_kw=dedupe(unique_keywords, threshold = 89)
ded_kw=dedupe(unique_keywords, threshold = 89)
print(len(ded_kw))

32704


In [ ]:
#write the deduplicated keywords into the file
#pd.Series(list(ded_kw)).to_csv('../data/interim/out_dedupl_10k_articles_only_100323.csv')
pd.Series(list(ded_kw)).to_csv('../data/interim/out_dedupl_2019-2021_articles_only_100323.csv')

In [ ]:
# # #read from csv isntead of running DEDUP
# #ded_kw=pd.read_csv('../data/interim/out_dedupl_10k_articles_only_100323.csv')
# ded_kw=pd.read_csv('../data/interim/out_dedupl_2019-2021_articles_only_100323.csv')

In [ ]:
deduplicated=set(ded_kw['0'])

In [ ]:
deduplicated

### Exploring ratio of similarity for individual  keywords

In [ ]:
process.extract('angela merkel',unique_keywords,limit=40)


In [ ]:
process.extract('planetary defense conference',unique_keywords,limit=40)


In [ ]:
process.extract('Chosen Soren',unique_keywords,limit=20)

In [ ]:
process.extract('Sex pistols',unique_keywords,limit=20)

In [ ]:
process.extract('UEFA',unique_keywords,limit=40)

In [ ]:
process.extract('UAE',unique_keywords,limit=40)

In [ ]:
process.extract('United Arab Emirates',unique_keywords,limit=40)

In [ ]:
process.extract('war in Ukraine',unique_keywords,limit=40)

In [ ]:
process.extract('UK',unique_keywords,limit=40)

In [ ]:
process.extract('United Kingdom',unique_keywords,limit=40)

In [ ]:
# #example from the fuzzywuzzy tutorial on token ratio
# Str1 = "The supreme court case of Nixon vs The United States"
# Str2 = "Nixon v. United States"
# Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
# Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
# Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
# Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)
# print(Ratio)
# print(Partial_Ratio)
# print(Token_Sort_Ratio)
# print(Token_Set_Ratio)


### "Putting back" merged clean keywords into the dataframe

In [ ]:
list_kw=list(unique_keywords)[0]

In [ ]:
df_subset['keywordStrings'][0]

In [ ]:
# i=0
# len(df_subset['keywordStrings'][i])

In [ ]:
# #n=0
# df_subset['keywordStrings'][i][n]

In [ ]:
# 1) loop over each line in dataframe
# 2) loop over each keyword in the line
# 3) find process.extractOne a substitute from deduplicated list
# 4) create a new column in dataframe with merged keywords


def put_clean_kw_into_df(dataframe):
    i = 0
    sample_line = []
    sample_ratio = []
    while i <= len(dataframe): # 10:
        print(i)
        n = 0
        line_wr = []
        ratio_line_wr = []
        #print(line_wr)
        while n < len(dataframe[i]):
            #print(process.extractOne(df_subset['keywordStrings'][i][n],deduplicated)) #print word and ratio 
            line_wr.append(process.extractOne(dataframe[i][n], deduplicated)[0])
            ratio_line_wr.append(process.extractOne(dataframe[i][n], deduplicated)[1])
            n = n+1

        sample_line.append(line_wr)
        sample_ratio.append(ratio_line_wr)

        i = i+1
    return sample_line
        
    
df_lines=put_clean_kw_into_df(df_subset['keywordStrings'])



In [ ]:
print(lines)

In [ ]:
df_subset['keywordStrings'][i]


In [ ]:
#figure out list comprehension later
# i=0
# def put_clean_kw_into_df(dataframe):
#     return [process.extractOne(dataframe[n], deduplicated)[0] for dataframe[n] in dataframe]

# put_clean_kw_into_df(df_subset['keywordStrings'][i])